In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import numpy as np
import random
import tensorflow as tf
from math import ceil
from glob import glob
from pysptk import sptk
from collections import defaultdict
from scipy.signal import get_window
from scipy import signal
import pandas as pd
import malaya_speech

In [3]:
def butter_highpass(cutoff, fs, order = 5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype = 'high', analog = False)
    return b, a

vctk = glob('vtck/**/*.flac', recursive = True)

vctk_speakers = defaultdict(list)
for f in vctk:
    s = f.split('/')[-1].split('_')[0]
    vctk_speakers[s].append(f)

files = glob('/home/husein/speech-bahasa/ST-CMDS-20170001_1-OS/*.wav')
speakers_mandarin = defaultdict(list)
for f in files:
    speakers_mandarin[f[:-9]].append(f)

df_nepali = pd.read_csv(
    '/home/husein/speech-bahasa/nepali_0/asr_nepali/utt_spk_text.tsv',
    sep = '\t',
    header = None,
)
asr_nepali = glob('/home/husein/speech-bahasa/*/asr_nepali/data/*/*.flac')
asr_nepali_replaced = {
    f.split('/')[-1].replace('.flac', ''): f for f in asr_nepali
}
df_nepali = df_nepali[df_nepali[0].isin(asr_nepali_replaced.keys())]

speakers_nepali = defaultdict(list)
for i in range(len(df_nepali)):
    speakers_nepali[df_nepali.iloc[i, 1]].append(
        asr_nepali_replaced[df_nepali.iloc[i, 0]]
    )

speakers = []
for s in vctk_speakers.keys():
    speakers.extend(
        random.sample(vctk_speakers[s], min(500, len(vctk_speakers[s])))
    )

for s in speakers_mandarin.keys():
    speakers.extend(
        random.sample(speakers_mandarin[s], min(300, len(speakers_mandarin[s])))
    )

for s in speakers_nepali.keys():
    speakers.extend(
        random.sample(speakers_nepali[s], min(300, len(speakers_nepali[s])))
    )

salina = glob('/home/husein/speech-bahasa/salina/output-wav-salina/*.wav')
salina = random.sample(salina, 5000)
male = glob('/home/husein/speech-bahasa/turki/output-wav-turki/*.wav')
male.extend(
    glob(
        '/home/husein/speech-bahasa/dari-pasentran-ke-istana/output-wav-dari-pasentran-ke-istana/*.wav'
    )
)
male = random.sample(male, 5000)
haqkiem = glob('/home/husein/speech-bahasa/haqkiem/*.wav')
khalil = glob('/home/husein/speech-bahasa/tolong-sebut/*.wav')
mas = glob('/home/husein/speech-bahasa/sebut-perkataan-woman/*.wav')
husein = glob('/home/husein/speech-bahasa/audio-wattpad/*.wav')
husein.extend(glob('/home/husein/speech-bahasa/audio-iium/*.wav'))
husein.extend(glob('/home/husein/speech-bahasa/audio/*.wav'))
husein.extend(glob('/home/husein/speech-bahasa/sebut-perkataan-man/*.wav'))

files = salina + male + haqkiem + khalil + mas + husein + speakers
sr = 22050
freqs = {'female': [100, 600], 'male': [50, 250]}
b, a = butter_highpass(30, sr, order = 5)


def speaker_normalization(f0, index_nonzero, mean_f0, std_f0):
    f0 = f0.astype(float).copy()
    f0[index_nonzero] = (f0[index_nonzero] - mean_f0) / std_f0
    f0[index_nonzero] = np.clip(f0[index_nonzero], -3, 4)
    return f0


def preprocess_wav(x):
    if x.shape[0] % 256 == 0:
        x = np.concatenate((x, np.array([1e-06])), axis = 0)
    y = signal.filtfilt(b, a, x)
    wav = y * 0.96 + (np.random.uniform(size = y.shape[0]) - 0.5) * 1e-06
    return wav


def get_f0(wav, lo, hi):
    f0_rapt = sptk.rapt(
        wav.astype(np.float32) * 32768, sr, 256, min = lo, max = hi, otype = 2
    )
    index_nonzero = f0_rapt != -1e10
    mean_f0, std_f0 = (
        np.mean(f0_rapt[index_nonzero]),
        np.std(f0_rapt[index_nonzero]),
    )
    return speaker_normalization(f0_rapt, index_nonzero, mean_f0, std_f0)


def pad_seq(x, base = 8):
    len_out = int(base * ceil(float(x.shape[0]) / base))
    len_pad = len_out - x.shape[0]
    assert len_pad >= 0
    return np.pad(x, ((0, len_pad), (0, 0)), 'constant'), x.shape[0]

In [4]:
len(files)

194817

In [5]:
!mkdir speechsplit-dataset

mkdir: cannot create directory ‘speechsplit-dataset’: File exists


In [6]:
from tqdm import tqdm
import pickle

def loop(files):
    files, index = files
    vggvox_v2 = malaya_speech.gender.deep_model(model = 'vggvox-v2')
    for i in tqdm(range(len(files))):
        f = files[i]
        try:
            x, fs = malaya_speech.load(f, sr = sr)
            wav = preprocess_wav(x)
            x_16k = malaya_speech.resample(x, sr, 16000)
            lo, hi = freqs.get(vggvox_v2(x_16k), [50, 250])
            f0 = np.expand_dims(get_f0(wav, lo, hi), -1)
            mel = malaya_speech.featurization.universal_mel(wav)
            with open(f'speechsplit-dataset/{index}-{i}.pkl', 'wb') as fopen:
                pickle.dump([wav, f0, mel], fopen)
        except:
            pass

In [7]:
import mp

mp.multiprocessing(files, loop, cores = 7, returned = False)

  0%|          | 38/27831 [00:10<1:59:51,  3.86it/s]/home/husein/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/husein/.local/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
  0%|          | 31/27831 [00:10<2:23:28,  3.23it/s]/home/husein/.local/lib/python3.6/site-packages/numpy/core/_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/husein/.local/lib/python3.6/site-packages/numpy/core/_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/husein/.local/lib/python3.6/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
  1%|          | 321/27831 [01:28<2:19:57,  3.28it/s]/home/husein/.local/li